In [1]:
#importing packages
import os
import pandas as pd


In [3]:
#This is the first function of this code. The input MUST be a path that contains ONLY files in a three string character, i.e. 044, 017, 113, 002 (no.tsv
#extension): Try:
#for file in *; do basename=$(echo $file | cut -d "_" -f 4); mv $file $basename; done;
#for file in *; do basename=$(echo $file | cut -d "." -f 1; mv $file $basename; done;
#Also make sure that the first five lines of the eggnog file are gone!
#for file in *; do sed -n '6,$p' $file > tmp/$file; done;

def adding_sample_name(my_path):
    os.chdir(my_path)
    for file in os.listdir(my_path):
        open_file = pd.read_table(file)
        sample_name = file[:3]
        open_file['sample_name'] = sample_name
        open_file['sample_name'] = open_file['sample_name'].astype(str)
        open_file.to_csv(str(file)+"_added",sep='\t', index=False)
        
#This function adds the sample name (three characters which are digits corresponding to the sample name)
#to the eggnog.tsv file

In [4]:
def removing_last_three_rows(my_path):
    os.chdir(my_path)
    for file in os.listdir(my_path):
        if file.endswith("_added"):
            dtype_mapping = {'sample_name':str}
            open_file = pd.read_table(file, dtype=dtype_mapping)
            df = open_file.iloc[:-3]
            df.to_csv(str(file)+"_tailed",sep='\t',index=False)
#This function removes the last three rows from the eggnog sheet, which corresponds to the time and query
#ran by eggnog.

In [5]:
def subsetting(my_path):
    for file in os.listdir(my_path):
        if file.endswith("_tailed"):
            dtype_mapping = {'sample_name':str}
            df = pd.read_table(file, dtype = dtype_mapping)
            new_df = df.iloc[:,[11,21]]#Change to 10 for EC, and 11 for KO
            new_df2 = df.iloc[:,[10,21]]#Change to 10 for EC, and 11 for KO
            new_df.to_csv(str(file)+"_cut_KO",sep='\t',index=False) #Change name for EC or KO
            new_df2.to_csv(str(file)+"_cut_EC",sep='\t',index=False)

#This function only takes the eleventh column and 21st column (Eleventh belongs to KO, 
#and 21st belongs to sample number just added). If wanted to change to EC, then select 10th column

In [6]:
def replacing_dashed(my_path):
    for file in os.listdir(my_path):
        if file.endswith("_cut_KO"):
            dtype_mapping = {'sample_name':str}
            open_file = pd.read_table(file, dtype = dtype_mapping)
            open_file['KEGG_ko'] = open_file['KEGG_ko'].replace("-","NA") #replaces dashes with NA
            open_file['KEGG_ko'] = open_file['KEGG_ko'].str.replace("ko:","") #removes 'ko' and leaves the KO###### number needed for picrust2
            open_file.to_csv(str(file)+"_replaced_KO",sep='\t',index=False, header=False)
        elif file.endswith("_cut_EC"):
            open_file = pd.read_table(file, dtype = dtype_mapping)
            open_file['EC'] = open_file['EC'].replace("-","NA") #replaces dashes with NA
            open_file.to_csv(str(file)+"_replaced_EC",sep='\t',index=False, header=False)

In [7]:
#In here, I did in unix: 'for file in *_replaced; do cat $file >> final_merged.tsv; done
#The pandas command to merge did not work for me, and it was making a mess

In [8]:
def delete_rows(file_KO,file_EC):
    dtype_mapping = {1:str}
    df = pd.read_table(file_KO, dtype = dtype_mapping)
    df2 = pd.read_table(file_EC, dtype = dtype_mapping)
    df_clean = df.dropna()
    df_clean2 = df2.dropna()
    df_clean.to_csv(str(file_KO),sep='\t', index=False, header=False)
    df_clean2.to_csv(str(file_EC), sep='\t', index=False, header=False)

In [9]:
def getting_final_file(file):
    dtype_mapping = {1:str} #makes sure to transformt the column of "integers" to strings.
    df_clean = pd.read_table(file, dtype = dtype_mapping)
    id_list=[]
    ko_list=[]
    id_list_tmp=[]
    ko_list_tmp=[]
    for item in df_clean.iloc[:,1]:
        id_list.append("seq"+str(item))
    for item in df_clean.iloc[:,0]:
        ko_list.append(item)
    for num, item in zip(id_list, ko_list):
        if ',' in item:
            parts = item.split(",")
            id_list_tmp.extend([num] * len(parts))
            ko_list_tmp.extend(parts)
        else:
            id_list_tmp.append(num)
            ko_list_tmp.append(item)
    final_df = pd.crosstab(index = id_list_tmp, columns = ko_list_tmp)
    if file.endswith("_KO.tsv"):
        final_df.to_csv("final_KO_list.tsv", sep='\t', index=True, header=True)
    elif file.endswith("_EC.tsv"):
        final_df.to_csv("final_EC_list.tsv", sep='\t', index=True, header=True)



In [15]:
def main():
    #path = "/Users/danielcm/Desktop/Sycuro/Projects/Diabetes/picrust2/eggnogs/metabolism_eggnog/raw_eggnogs/"
    #adding_sample_name(path)
    #removing_last_three_rows(path)
    #subsetting(path)
    #replacing_dashed(path)
    file_ko = "/Users/danielcm/Desktop/Sycuro/Projects/Diabetes/picrust2/eggnogs/metabolism_eggnog/raw_eggnogs/merged_ko/merged_KO.tsv"
    file_ec = "/Users/danielcm/Desktop/Sycuro/Projects/Diabetes/picrust2/eggnogs/metabolism_eggnog/raw_eggnogs/merged_ec/merged_EC.tsv"
    delete_rows(file_ko,file_ec)
    getting_final_file(file_ko)
    getting_final_file(file_ec)
    #After this, just rename the very first cell to 'assembly' (I did this manually)

In [16]:
main()

In [ ]:
#TO DO:
#MAKE SURE THAT THE CODE IS FULLY AUTOMATED: ITS INPUT ARE EGGNOG FILES IN THREE DIGIT FORMAT AND OUTPUT IS FINAL FILE WITH THE 'ASSEMBLY'
#WORD ON TOP. MAKE SURE YOU DON'T HAVE TO RUN ANY CODE IN BASH FOR IT TO WORK!